# Dynamic Programming

## Purpose

The simplest way to think about dynamic programming, "DP", is nothing more than as an approach to optimize an algorithm:

1.  Find an algorithm that "brute forces" a problem
2.  Notice that it fits into the the class of algorithms DP can optimize
3.  Implement algorithm using DP

Dynamic programming does **not** have anything to do with on-the-fly code generation, online reinforcement learning, editing/optimizing compiled code while it's running... or anything else fancy.  So what does it do?  Read on!

## Approach

So how does DP work?

It boils down to a simple idea:  _don't let your code do the same work twice_.  In practice, this means that you should cache intermediate results (say, in a table).  Each time you need to evaluate a function with the same input after the first, you can look up the result in the cache rather than recomputing it.

You will sometimes hear DP is used for problems whose solutions are combinations of the solutions to "optimal subproblems".  Unpacking this a little bit:  if each solution is just a combination of smaller versions of the same problem, and you cache the results of the smaller problems as you go, to the extent that you need those solutions in more than one place, you'll get a speed up.  For many problems, an exponential time naive implementation can be made to run in polynomial time.

**Don't let this sound complicated to you.**  All we're doing is adding a cache to avoid recomputing an "expensive" function.  Any time you see a problem that sounds like, "Try every possible combination of X and pick the best", you should think "I wonder if there are any partial results here we can cache and reuse".

## Why is this important for natural language processing?

The last half of the course contains a wide variety of algorithms that run passibly fast only when we apply a judicious amount of dynamic programing. Notably, both sequence tagging tasks (like part-of-speech tagging and entity recognition) and structured-prediction tasks like parsing will make heavy use of DP!

## Example: Climbing the stairs

Let's say you want to compute the total number of ways you can run up a flight of stairs.  At each step, $k$, you can either:

- Take a regular step up to $k + 1$
- Take a big step up to $k + 2$.

Starting on stair $0$ (the bottom floor), how many unique ways are there to get up a staircase with $n$ stairs?

If you've not encountered this kind of problem before, **spend a minute pondering** how you might compute this.  It's common to feel "lost" in the combinatorial explosion of options.

### Hint

Could you figure out the number of ways up a staircase of height $n$ if you were told how many ways you might get up staircases of height $n - 1$ and $n - 2$?  That is, if you knew the solution to a _subproblem_, could you use that to compute the solution to your real problem?

### Solution

If there are $ways(n - 1)$ to get to the $n - 1$ stair, you can use any of those methods to get there and then take a regular step to get to stair $n$.  Similarly, if you have $ways(n - 2)$ to get to the $n - 2$ step and then take a big step to stair $n$.  There is no other way to get to stair $n$ except by one of those two sets of options.  There is no overlap between those sets of sequences as sequences in the first set always end in a "regular" step and those sequences in the second set always end in a "big" step.

In math, $ways(n) = ways(n-1) + ways(n-2)$.

Also notice (imagine, or draw a picture), $ways(1) = 1$ and $ways(2) = 2$.

*Aside: you may recognize this as the famous [Fibbonaci series](https://en.wikipedia.org/wiki/Fibonacci_number).*

The next cell shows one implementation.

In [1]:
def naive_ways(n):
    """Return the ways up a staircase of length n. Uses a naive algorithm."""
    if n == 1:
        return 1
    if n == 2:
        return 2
    
    return naive_ways(n - 1) + naive_ways(n - 2)

naive_ways(3)

**Great!**

Unfortunately, this implementation gets very slow for large $n$. The cell below will print some timing information.

In [2]:
# TIMING INFO FOR PART A
for n in range(20, 25+1):
    print ("n=%d: " % n),
    %timeit -n25 naive_ways(n)

In [3]:
# Try with slightly-larger n.
%timeit -n1 naive_ways(35)

If we try to handle a staircase with 100 steps, it'll take a _very_ long time with our naive implementation.

## Part (a): Short Answer Questions

Give *brief* answers to the following in the cell below, then for 4., implement the DP version in the cell below.

1. Based on the timing numbers from the `# TIMING INFO FOR PART A` cell above, approximately how much slower is $ways(n)$ than $ways(n-1)$? (i.e. what is $\frac{time(ways(n))}{time(ways(n-1))}$, roughly?)

2.  Why is this so slow?  What calculations do we compute repeatedly?  Hint: consider the following diagram.
![DP diagram](dp.png)

3. **Food for thought (not graded):** Assuming that $time(ways(n)) = time(ways(n-1)) + time(ways(n-2))$, what is $\lim_{n\to\infty} \frac{time(ways(n))}{time(ways(n-1))}$?

4. To remove the duplication you found in 2., we apply DP.  Remember DP just means that we cache the results of the subproblems so we don't have to keep recomputing them.  Or, more directly, it prevents the duplication you noted in your answer above. Finish the implementation of the **`table_ways()`** function below.

## Part (a) Answers
1.  _Your answer goes here!_
2.  _Your answer goes here!_
3.  _Your answer goes here!_
4.  _(Write code in cell below.)_

### 4. Apply Dynamic Programming

In [4]:
def table_ways(n, cache=None):
    """Return the ways up a staircase of length n. Uses a DP algorithm."""
    # Hold a mapping from the parameter "n" to the answer table_ways(n).
    # With "simple" integer keys, we could just use an array here, but
    # we'll use a dict to reinforce that the DP cache can be keyed by
    # any type.
    if cache is None:
        cache = {1:1, 2:2}

    #### YOUR CODE HERE ####
    #  Check the cache to see if already have an answer to table_ways(n).
    #  If it's already there, return it.
    
    
    #### END(YOUR CODE) ####
    
    result = table_ways(n - 1, cache) + table_ways(n - 2, cache)
    
    #### YOUR CODE HERE ####
    #  Store the result back into the cache.
    
    #### YOUR CODE HERE ####
    
    return result

assert table_ways(1) == 1
assert table_ways(2) == 2
assert table_ways(100) == 573147844013817084101L
%time table_ways(100)

If implemented correctly, you should get the answer almost instantly for $n=100$ - certainly much faster than the naive version on a much smaller problem!

That is the power of DP.

## Part (b): Explicit order of computation

At the end of the execution above, we have a complete cache for all inputs from $1$ to $n$.  By carefully planning the order in which we try to evaluate f(1), f(2), ..., f(n), it's possible to write the above algorithm keeping track of only two cache values at a time.

First answer the following questions, then use what you gleaned to write the code.

### Short Answer Questions
1. What are the values of A, B, and C in this table?
<html><table>
<tr><td>n</td><td>ways(n)</td></tr>
<tr><td>1</td><td>1</td></tr>
<tr><td>2</td><td>2</td></tr>
<tr><td>3</td><td>3</td></tr>
<tr><td>4</td><td>5</td></tr>
<tr><td>5</td><td>8</td></tr>
<tr><td>6</td><td>13</td></tr>
<tr><td>7</td><td>$A$</td></tr>
<tr><td>8</td><td>$B$</td></tr>
<tr><td>9</td><td>$C$</td></tr>
</table>
</html>
<p>
2. To compute these values, did you look at $n=4$ or earlier?
<p>
3. What is the minimum number of values you need to keep as you fill the table from top to bottom, while maintaining the DP property of not recomputing any values?  
<p>

4. Write an implementation that implements a DP solution while storing the minimum amount of data. Implement the **`minimum_ram_ways()`** function in the cell below.

### Part (b) Answers

- 1A = _Your answer here_
- 1B = _Your answer here_
- 1C = _Your answer here_
- 2 _Your answer here_
- 3 _Your answer here_
- 4 _(Write code in cell below.)_

In [5]:
def minimum_ram_ways(n):
    """Return the ways up a staircase of length n. Uses a low-RAM DP algorithm."""
    pass
    # YOUR CODE HERE




    # END YOUR CODE

assert minimum_ram_ways(1) == 1
assert minimum_ram_ways(2) == 2
assert minimum_ram_ways(100) == 573147844013817084101L

%time minimum_ram_ways(100)

## Part (c): Pipe cutting

Another problem suceptible to optimization with DP is pipe cutting.  Here, we are given a metal pipe of length $n$ and a price list, the price at which you can sell various parts of the pipe.  For the purposes of this exercise, let's write the code as though different parts of the pipe could yield different amounts of money, even if they're the same length.  Your objective is to cut the pipe into pieces such that they fetch the most revenue.

Implement the code to determine the maximum revenue you can achieve from such a pipe.

### Part (c) tasks
- Implement the **`best_cuts()`** function in the cell below.

### Hints

- Iterate over the possible locations of the first cut, then recursively call yourself to optimally split the bar to the right of the cut.

- You only need to recursively call yourself to the right of the cut (just call the revenue function for the left hand side).  Calling yourself recursively on the left would just duplicate the work you already did when you tried a "first cut" further to the left of your current position.

- The cache should be a map from a tuple `(left, right)` to a float denoting the best revenue you can get for that section of pipe (including any cuts you might choose to make within that span).

- Don't worry about carefully planning the execution order like you did in the last section.  Just get it to work (cache everything).

- Python's **`xrange`** function can be used with both a start and an end. For example:
```python
for i in xrange(3, 8):
    do_stuff(i)  # i = 3, 4, 5, 6, 7
```
You may find this useful for iterating over the location of the cut.

### Why do we care?

It turns out the pipe cutting problem is (very) closely related to DP algorithms for segmentation in NLP, as we'll see in more detail below.

In [6]:
price_list = [0.0, 3.4, 5.7, 17.6, 2.2, 86.3]
def pipe_revenue_function(left, right):
    """$ revenue for the part of the bar in interval [left, right)."""
    n = right - left
    if n < 0 or n >= len(price_list):
        return 0.0
    return price_list[n]

# DP cache for the function below.
def best_cuts(left, right, score, cache=None):
    """Determine the optimal revenue possible by (optionally) cutting a bar
       of length right - left.
    
    best_cuts(0, 9) is 107.30 with bars of length 1, 3, and 5.
    The return value will be: 107.3

    Args:
      left: the left index of the pipe piece to consider cutting
      right: the right index of the pipe piece to consider cutting
      score: a function that accepts "left" and "right and gives you the score
             (revenue) received for the segment of pipe extending the interval
             from [left, right)
      cache: a dict mapping tuples (left, right) to the best_score for that interval
    
    Returns:
      The best revenue for the span of pipe [left, right)
      
    HINT: You *must* call the "score" function passed in, not the pipe_revenue_function
          directly.  In a cell further down, we're going to pass in a different
          function instead!
    """
    if cache is None:
        cache = {}
        
    #### YOUR CODE HERE ####

        




    ##### END(YOUR CODE) ####
    
assert 3.4 == best_cuts(0, 1, pipe_revenue_function)
assert 107.3 == round(best_cuts(0, 9, pipe_revenue_function), 1)

round(best_cuts(0, 9, pipe_revenue_function), 2)

## Part (d): Bookkeeping

Great!  We know we can make over a hundred dollars cutting up our length 9 bar!

Unfortunately, while we computed the revenue available by cutting the bar, we didn't actually track the cuts we need to make in order to earn it!

Write the code (in the cell below) to keep track of the steps you took in order to achieve the optimal revenue.  **Start by copying and pasting your solution from above and work from there.**

**Hint:** the cache currently maps from bar span, `(left, right)`, to optimal revenue that can be made from that section.  In this part, keep the key the same but instead map it to a tuple of `(optimal revenue, [list, of, cut, positions])`.

### Part (d) tasks:
- Implement the **`best_cuts_with_trace()`** function below.

In [10]:
price_list = [0.0, 3.4, 5.7, 17.6, 2.2, 86.3]
def pipe_revenue_function(left, right):
    """$ revenue for the part of the bar in interval [left, right)."""
    n = right - left
    if n < 0 or n >= len(price_list):
        return 0.0
    return price_list[n]

# DP cache for the function below.
def best_cuts_with_trace(left, right, score, cache=None):
    """Determine the optimal revenue possible by (optionally) cutting a bar
       of length right - left.
    
    best_cuts(0, 9) is 107.30 with bars of length 1, 3, and 5 (cuts at positions 1 and 4).
    The return value will be: (107.3, [1, 4])

    Args:
      left: the left index of the pipe piece to consider cutting
      right: the right index of the pipe piece to consider cutting
      score: a function that accepts "left" and "right and gives you the score
             (revenue) received for the segment of pipe extending the interval
             from [left, right)
      cache: a dict mapping tuples (left, right) to a tuple (best_revenue, [list, of, cuts])
    
    Returns:
      Tuple (best_revenue, [list, of, cuts])
      
    HINT: As above, you *must* call the "score" function passed in, not the
          pipe_revenue_function directly.  In a cell further down, we're
          going to pass in a different function instead!
    """
    if cache is None:
        cache = {}
        
    #### YOUR CODE HERE ####

        




    ##### END(YOUR CODE) ####
    
assert round(best_cuts_with_trace(0, 9, pipe_revenue_function)[0], 1) == 107.3
cuts = sorted(best_cuts_with_trace(0, 9, pipe_revenue_function)[1])
cut_sizes = sorted([x[1] - x[0] for x in zip([0] + cuts, cuts + [9])])
print 'Cuts:', cuts, 'Bar sizes:', cut_sizes
assert cut_sizes == [1, 3, 5]

best_cuts_with_trace(0, 9, pipe_revenue_function)

We'll see bookkeeping like this throughout the rest of the course.  For example, we'll want to know the optimal way to tag words in a sentence with their parts of speech.  The optimization will be over some likelihood of a particular assignment (rather than revenue).  This optimization is only useful to us however if we have a way to know what sequence of part of speech tags gave us that score, so we'll have to do this same kind of bookkeeping.

## Pipe cutting is segmentation
An astute reader may have realized that we've actually seen this optimal splitting algorithm before.  Instead of bars, we split up a long run of text into words in [week 2](../../materials/week2/lm1.ipynb) (search for "segment").  In that case, ["score"](../../materials/week2/segment.py#L24) was the logarithm of the unigram probability of the word (we took the logarithm so that adding up the "score" is equivalent to multiplying the probabilities).

The next cell implements a very light wrapper around your best_cut function (it just takes the cut indexes your function returns and turns them into text to pretty-print).  It also implements a simple unigram language model (much simpler than what you implemented in A2, let alone A3!).

Experiment with some sentences.  See if you can find at least one that breaks it (this shouldn't be hard).

**Hints**:

If the results below aren't very good, go back and do a careful check on the initialization you did up above.  If you initialize your "best_cuts" to an empty list (no cuts), you must be careful to initialize your best_score to the corresponding value (not a hard-coded 0.0 - or, for that matter, any other hard coded value!).  Remember that segmentation is using the log(P(unigram)) as the scoring function and is therefore <= 0!  0.0 will look very good in comparison!

## Question
1.  What is the right value to initialize your score to if you've initialized your cuts list to []?

## Answer
1.  _Your answer to question 1._

In [8]:
import numpy as np

##
# Compute unigram counts from a simple corpus.
unigram_counts = {}
total_counts = 0
for line in open('english_uni_simplified_sorted_top').readlines():
    word_and_count = line.split()
    word = word_and_count[0].strip('"')
    count = int(word_and_count[1])
    unigram_counts[word] = count
    total_counts += count

def unigram_scoring_function(text, left, right):
    word = text[left:right]
    if word in unigram_counts:
        # Log probabilities, so we can add scores instead of multiplying
        return np.log(unigram_counts[word]) - np.log(total_counts)
    else:  
        # "Smoothing", encouraging in-vocabulary, or at least short OOV words.
        # We give a lower score to longer out-of-vocabulary spans.
        return -100 * (right - left)
        
##
# Use the pipe-cutting algorithm to segment text.
def segment(text):
    # We create a scoring lambda that accepts two parameters, "left" and "right", as required by the
    # code you implemented above.  However, we also need access to the "text" in order
    # to score the unigram.  A lambda captures the local variable "text" for this purpose.
    score_func = lambda left, right: unigram_scoring_function(text, left, right)
    
    # Call your function to slice the string.
    score, cuts = best_cuts_with_trace(0, len(text), score_func)
    
    # Imply a "cut" at the start and end of the text so that the list comprehension below is convenient.
    cuts = [0] + cuts + [len(text)]
    
    # Convert the list of cuts into a list of words.
    return score, [text[cuts[i] : cuts[i + 1]] for i in xrange(len(cuts) - 1)]

In [9]:
segment('helloworldhowareyou')

In [10]:
segment('downbythebay')

In [11]:
segment('wikipediaisareallystrongresourceontheinternet')

## Part (e): String edit distance
Another classic DP problem in the NLP space - but not one we otherwise will talk about in the course is the idea of "[edit distance](https://en.wikipedia.org/wiki/Levenshtein_distance)".  It's a way of measuring how many "edits" to one string you need to make in order to turn it into another.

We've provided two implementations below for you to play with.

1.  **levenshtein_cache:** The "cache everything in a dict" approach is first.  The keys are coordinates into a table that is len(str1) x len(str2) in size.

2.  **levenshtein_explicit:** Similar to the version of ways(n) that only keeps the previous two values at hand, the explicit ordering approach only keeps the immediately previous row of the table while building the next. Setting the verbose flag to this version prints each row of the table out as it computes it.

### Part (e) Short Answer Questions:

Give brief answers to the following in the cell below.

1. Let `n = len(str1)` and `m = len(str2)`. In terms of `n` and `m`, what is the size of the DP table (cache) for computing Levenshtein distance? _Hint: how many valid keys are there? Do we use all of them?_
<p>
2. Based on your answer to 1., what is the running time (in Big-O notation) of the edit distance algorithm? _Hint: it takes $O(1)$ work at each step, assuming we have the needed cache entries._
<p>

3. Consider transpositions (as mentioned in section 5.6 of the async), such as `xy` -> `yx`. How can we compose a transposition from insertions, deletions, and substitutions? What is the edit distance between `wxyz` and `wyxz`?
<p>

4. **Optional (0 points, but we'll provide feedback when we grade):** Suppose we wanted to handle transpositions directly, rather than allowing our algorithm to compose them from other operations. (This might be useful if we want to score them differently.) If we have for the other operations:
```python
_ed(i - 1, j) + 1  # insertion
_ed(i, j - 1) + 1  # deletion
_ed(i - 1, j - 1) + substitution  # substitution, free if letters match  
```
what line would we add (calling `_ed`) to handle a transposition? (You may want to define a variable `transposition_match` to check that a transposition makes sense at the current position.) Based on your answer to 1. and 2., does this change the Big-O runtime of the algorithm?

### Part (e) Answers

1. _Your answer here!_
2. _Your answer here!_
3. _Your answer here!_
4. _Your answer here, optionally!_

In [12]:
def levenshtein_cache(str1, str2):
    cache = dict()
    def _ed(i, j):
        """Recursive helper, using cache."""
        if (i,j) in cache: 
            return cache[(i,j)]
        
        # Base cases
        if i == 0:
            result = j
        elif j == 0:
            result = i
            
        # Main recursion
        else:
            # 1 if letters differ (substitution is free if the letters are the same)
            substitution = 0 if str1[i - 1] == str2[j - 1] else 1
            result = min([
                    _ed(i - 1, j) + 1,  # insertion
                    _ed(i, j - 1) + 1,  # deletion
                    _ed(i - 1, j - 1) + substitution  # substitution, free if letters match  
            ])
        cache[(i,j)] = result
        return result
    
    return _ed(len(str1), len(str2))

In [13]:
def levenshtein_explicit(str1, str2, verbose=False):
    prev_num_edits = range(len(str1) + 1)
    for j in xrange(1, len(str2) + 1):
        num_edits = [prev_num_edits[0] + 1]
        for i in xrange(1, len(str1) + 1):
            # 1 if letters differ (substitution is free if the letters are the same)
            substitution = 0 if str1[i - 1] == str2[j - 1] else 1
            result = min([num_edits[i - 1] + 1,
                          prev_num_edits[i] + 1,
                          prev_num_edits[i - 1] + substitution
            ])
            num_edits.append(result)
        if verbose:
            print prev_num_edits
        prev_num_edits = num_edits
    if verbose:
        print prev_num_edits
    return prev_num_edits[len(str1)]

In [14]:
# Substitution.
levenshtein_explicit('abc', 'dbc', verbose=True)

In [15]:
# Deletion.
levenshtein_explicit('abc', 'ac')

In [16]:
# Insertion.
levenshtein_explicit('ac', 'abc')

In [17]:
# All of the above.
levenshtein_cache('kitten', 'sitting')

In [18]:
# Fun!
levenshtein_cache('w266 class', 'with 6 classic tricks')